In [19]:
import os,sys,gc,pickle
from pathlib import Path
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split, GroupKFold
sys.path.append("../script_lgb/")
from utils import utils, feature_util
import compe_data

In [24]:
train_df = compe_data.read_train()

In [25]:
train_df.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074


In [26]:
gnn_oof = pd.read_csv("../data/oof_pred1.csv")

In [27]:
gnn_oof.head()

,id,scalar_coupling_constant
0,19,83.378815
1,20,-2.054809
2,21,-11.366610
3,22,-11.365542
4,23,12.175028


In [28]:
train_df = train_df.merge(gnn_oof.rename(columns={"scalar_coupling_constant":"oof_pred"}), on="id", how="left")

In [29]:
train_df.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,oof_pred
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,NaN
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570,NaN
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548,NaN
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543,NaN
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074,NaN


In [30]:
train_df["target"] = np.abs(train_df["scalar_coupling_constant"] - train_df["oof_pred"])

In [31]:
del train_df["scalar_coupling_constant"], train_df["oof_pred"]
gc.collect()

35

In [32]:
train_df.head()

,id,molecule_name,atom_index_0,atom_index_1,type,target
0,0,dsgdb9nsd_000001,1,0,1JHC,NaN
1,1,dsgdb9nsd_000001,1,2,2JHH,NaN
2,2,dsgdb9nsd_000001,1,3,2JHH,NaN
3,3,dsgdb9nsd_000001,1,4,2JHH,NaN
4,4,dsgdb9nsd_000001,2,0,1JHC,NaN


## 特徴量を読み込む

In [33]:
ls ../feature/train/

ACSFDescriptor.pkl           GroupAtomIndex1.pkl
A_ACSFDescriptor.pkl         HinokkiMeta.pkl
A_AtomBasicFeat.pkl          MakeDistance.pkl
A_AtomInfoFromGraphData.pkl  MergeAtomInfo.pkl
A_BondInfo.pkl               MoleculeFeat.pkl
AtomBondFeat.pkl             MullikenTotal.pkl
AtomInfoFromGraphData.pkl    NBondFeat.pkl
AtomMetaFeat.pkl             NearDistFeat.pkl
BondAngleFeat.pkl            NearON.pkl
BondAngleFeat012.pkl         NextACSFDescriptor.pkl
BondAngleFeat123.pkl         NextBondPerType.pkl
BondFeat.pkl                 OpenBabelMC.pkl
BondTypesFeat.pkl            PerTypeFeat.pkl
ClosestAndFarthest.pkl       SomeGroupDist.pkl
CompareOtherAtoms.pkl        SortedDist.pkl
CountFeature.pkl             TypeFeat.pkl
Feature_1JHC.pkl             TypeMapping.pkl


In [34]:
f_path = Path("../feature/train/")

In [37]:
len(train_df)

4658147

### make_distance

In [35]:
feat = pd.read_pickle(f_path / "MakeDistance.pkl")

In [38]:
len(feat)

4658147

In [36]:
feat.head()

,dist,inv_sum_dist_0,inv_sum_dist_1,inv_sum_dist_PR
0,1.091953,-5110.036611,-5110.036611,-2555.018305
1,1.783120,-5110.036611,-5110.036611,-2555.018305
2,1.783147,-5110.036611,-5110.036611,-2555.018305
3,1.783157,-5110.036611,-5110.036611,-2555.018305
4,1.091952,-5110.589581,-5110.589581,-2555.294790


In [39]:
train_df["dist"] = feat["dist"]

### MergeAtomInfo

In [43]:
feat = pd.read_pickle(f_path / "MergeAtomInfo.pkl")

In [44]:
feat.head()

,EN_0,EN_1,atom_0,atom_1,bond_lengths_1_0,bond_lengths_1_1,bond_lengths_2_0,bond_lengths_2_1,bond_lengths_3_0,bond_lengths_3_1,...,n_bonds_0,n_bonds_1,rad_0,rad_1,x_0,x_1,y_0,y_1,z_0,z_1
0,2.2,2.55,H,C,1.091953,1.091946,NaN,1.091948,NaN,1.091952,...,1,4,0.43,0.82,0.002150,-0.012698,-0.006031,1.085804,0.001976,0.008001
1,2.2,2.20,H,H,1.091953,1.091952,NaN,NaN,NaN,NaN,...,1,1,0.43,0.43,0.002150,1.011731,-0.006031,1.463751,0.001976,0.000277
2,2.2,2.20,H,H,1.091953,1.091946,NaN,NaN,NaN,NaN,...,1,1,0.43,0.43,0.002150,-0.540815,-0.006031,1.447527,0.001976,-0.876644
3,2.2,2.20,H,H,1.091953,1.091948,NaN,NaN,NaN,NaN,...,1,1,0.43,0.43,0.002150,-0.523814,-0.006031,1.437933,0.001976,0.906397
4,2.2,2.55,H,C,1.091952,1.091946,NaN,1.091948,NaN,1.091952,...,1,4,0.43,0.82,1.011731,-0.012698,1.463751,1.085804,0.000277,0.008001


In [45]:
feat.head()

,EN_0,EN_1,atom_0,atom_1,bond_lengths_1_0,bond_lengths_1_1,bond_lengths_2_0,bond_lengths_2_1,bond_lengths_3_0,bond_lengths_3_1,...,n_bonds_0,n_bonds_1,rad_0,rad_1,x_0,x_1,y_0,y_1,z_0,z_1
0,2.2,2.55,H,C,1.091953,1.091946,NaN,1.091948,NaN,1.091952,...,1,4,0.43,0.82,0.002150,-0.012698,-0.006031,1.085804,0.001976,0.008001
1,2.2,2.20,H,H,1.091953,1.091952,NaN,NaN,NaN,NaN,...,1,1,0.43,0.43,0.002150,1.011731,-0.006031,1.463751,0.001976,0.000277
2,2.2,2.20,H,H,1.091953,1.091946,NaN,NaN,NaN,NaN,...,1,1,0.43,0.43,0.002150,-0.540815,-0.006031,1.447527,0.001976,-0.876644
3,2.2,2.20,H,H,1.091953,1.091948,NaN,NaN,NaN,NaN,...,1,1,0.43,0.43,0.002150,-0.523814,-0.006031,1.437933,0.001976,0.906397
4,2.2,2.55,H,C,1.091952,1.091946,NaN,1.091948,NaN,1.091952,...,1,4,0.43,0.82,1.011731,-0.012698,1.463751,1.085804,0.000277,0.008001


In [46]:
train_df = pd.concat([train_df, feat[[c for c in feat.columns if c not in 
                                      ["EN_0","EN_1","atom_0","atom_1","x_0","x_1","y_0","y_1","z_0","z_1"]
                                ]]], axis=1)

### TypeMapping

In [47]:
feat = pd.read_pickle(f_path / "TypeMapping.pkl")

In [48]:
feat.head()

,type_0,type_1,type_int
0,1,0,0
1,2,2,2
2,2,2,2
3,2,2,2
4,1,0,0


In [49]:
train_df = utils.fast_concat(train_df, feat)

### HinokkiMeta

In [50]:
feat = pd.read_pickle(f_path / "HinokkiMeta.pkl")

In [51]:
feat.head()

,new_type
0,0
1,1
2,1
3,1
4,0


In [52]:
train_df["new_type"] = feat["new_type"]

### NextBondPerType

In [53]:
feat = pd.read_pickle(f_path / "NextBondPerType.pkl")

In [54]:
feat.head()

,CC_next_L2dist_max,CC_next_L2dist_mean,CC_next_L2dist_min,CC_next_L2dist_std,CC_next_aromatic_sum,CC_next_double_sum,CC_next_single_sum,CC_next_triple_sum,CN_next_L2dist_max,CN_next_L2dist_mean,...,CO_next_single_sum,CO_next_triple_sum,all_next_L2dist_max,all_next_L2dist_mean,all_next_L2dist_min,all_next_L2dist_std,all_next_aromatic_sum,all_next_double_sum,all_next_single_sum,all_next_triple_sum
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.091953,1.091950,1.091946,0.000003,0.0,0.0,4.0,0.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.091952,1.091952,1.091952,NaN,0.0,0.0,1.0,0.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.091946,1.091946,1.091946,NaN,0.0,0.0,1.0,0.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.091948,1.091948,1.091948,NaN,0.0,0.0,1.0,0.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.091953,1.091950,1.091946,0.000003,0.0,0.0,4.0,0.0


In [57]:
train_df = utils.fast_concat(train_df, feat)

### MoleculeFeat

In [58]:
feat = pd.read_pickle(f_path / "MoleculeFeat.pkl")

In [59]:
feat.head()

,bond_count_C,bond_count_F,bond_count_H,bond_count_N,bond_count_O,count_C,count_F,count_H,count_N,count_O,...,rel_molecule_dist,rel_std_dist_in_index_atom_0,rel_std_dist_in_index_atom_1,size_0_0,size_0_1,size_1_0,size_1_1,std_dist_in_index_atom_0,std_dist_in_index_atom_01,std_dist_in_index_atom_1
0,4.0,NaN,6.0,NaN,NaN,1.0,NaN,4,NaN,NaN,...,0.724747,3.159640,341642.921024,4.0,NaN,NaN,4.0,0.345594,0.172799,0.000003
1,4.0,NaN,6.0,NaN,NaN,1.0,NaN,4,NaN,NaN,...,1.183486,5.159578,NaN,4.0,3.0,NaN,1.0,0.345594,0.345594,NaN
2,4.0,NaN,6.0,NaN,NaN,1.0,NaN,4,NaN,NaN,...,1.183504,5.159658,248119.985348,4.0,2.0,NaN,2.0,0.345594,0.172801,0.000007
3,4.0,NaN,6.0,NaN,NaN,1.0,NaN,4,NaN,NaN,...,1.183510,5.159685,360640.844342,4.0,1.0,NaN,3.0,0.345594,0.172800,0.000005
4,4.0,NaN,6.0,NaN,NaN,1.0,NaN,4,NaN,NaN,...,0.724746,2.736273,341642.470164,3.0,NaN,1.0,4.0,0.399065,0.199534,0.000003


In [61]:
train_df = utils.fast_concat(train_df, feat)

### SortedDist

In [62]:
feat = pd.read_pickle(f_path / "SortedDist.pkl")

In [63]:
feat.head()

,C_dist1,C_dist2,C_dist3,C_dist4,H_dist1,H_dist2,H_dist3,H_dist4,N_dist1,N_dist2,N_dist3,N_dist4,all_dist1,all_dist2,all_dist3,all_dist4
0,1.091953,NaN,NaN,NaN,1.783120,1.783147,1.783157,NaN,NaN,NaN,NaN,NaN,1.091953,1.783120,1.783147,1.783157
1,1.091953,NaN,NaN,NaN,1.783120,1.783147,1.783157,NaN,NaN,NaN,NaN,NaN,1.091953,1.783120,1.783147,1.783157
2,1.091953,NaN,NaN,NaN,1.783120,1.783147,1.783157,NaN,NaN,NaN,NaN,NaN,1.091953,1.783120,1.783147,1.783157
3,1.091953,NaN,NaN,NaN,1.783120,1.783147,1.783157,NaN,NaN,NaN,NaN,NaN,1.091953,1.783120,1.783147,1.783157
4,1.091952,NaN,NaN,NaN,1.783148,1.783158,NaN,NaN,NaN,NaN,NaN,NaN,1.091952,1.783148,1.783158,NaN


In [64]:
train_df = utils.fast_concat(train_df, feat)

In [65]:
train_df.shape

(4658147, 139)

In [67]:
train_df.head()

,id,molecule_name,atom_index_0,atom_index_1,type,target,dist,bond_lengths_1_0,bond_lengths_1_1,bond_lengths_2_0,...,H_dist3,H_dist4,N_dist1,N_dist2,N_dist3,N_dist4,all_dist1,all_dist2,all_dist3,all_dist4
0,0,dsgdb9nsd_000001,1,0,1JHC,NaN,1.091953,1.091953,1.091946,NaN,...,1.783157,NaN,NaN,NaN,NaN,NaN,1.091953,1.783120,1.783147,1.783157
1,1,dsgdb9nsd_000001,1,2,2JHH,NaN,1.783120,1.091953,1.091952,NaN,...,1.783157,NaN,NaN,NaN,NaN,NaN,1.091953,1.783120,1.783147,1.783157
2,2,dsgdb9nsd_000001,1,3,2JHH,NaN,1.783147,1.091953,1.091946,NaN,...,1.783157,NaN,NaN,NaN,NaN,NaN,1.091953,1.783120,1.783147,1.783157
3,3,dsgdb9nsd_000001,1,4,2JHH,NaN,1.783157,1.091953,1.091948,NaN,...,1.783157,NaN,NaN,NaN,NaN,NaN,1.091953,1.783120,1.783147,1.783157
4,4,dsgdb9nsd_000001,2,0,1JHC,NaN,1.091952,1.091952,1.091946,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.091952,1.783148,1.783158,NaN


## 学習

In [68]:
train_df = train_df[~train_df["target"].isna()]

In [79]:
train_df.reset_index(drop=True, inplace=True)

In [69]:
print(train_df.shape)
gc.collect()

(1552716, 139)


95

In [70]:
fold = GroupKFold(n_splits=3)

In [95]:
use_cols = [c for c in train_df.columns if c not in 
            ["target","id","molecule_name","atom_index_0","atom_index_1","type","type_0","type_1","c_x","c_y","c_z"]]

In [91]:
params = {
            'num_leaves': 40,
            'min_data_in_leaf': 30, 
            'objective':'regression',
            'max_depth': 4,
            "metric":"mae",
            'learning_rate': 0.05,
            "boosting": "gbdt",
            "feature_fraction": 0.8,
            "bagging_freq": 1,
            "bagging_fraction": 0.8 ,
            "bagging_seed": 11,
            "lambda_l1": 0.1,
            "lambda_l2": 0.1,
            "seed": 201,
            "verbosity": -1,
            "num_threads": 4,
            #"max_bin": 128,
}


In [96]:
def learn(df, train_type):
    train = df[df.type == train_type].reset_index(drop=True)
    for trn_, val_ in fold.split(train, train["target"], groups=train["molecule_name"]):

        trn_data = lgb.Dataset(
            train.loc[trn_, use_cols],
            label = train.loc[trn_, "target"]
        )
        val_data = lgb.Dataset(
            train.loc[val_, use_cols],
            label = train.loc[val_, "target"]
        )
        clf = lgb.train(
            params,
            trn_data,
            1000,
            valid_sets = [trn_data, val_data],
            verbose_eval=100,
            early_stopping_rounds =100,

        )

        oof_preds = clf.predict(train.loc[val_,use_cols] , num_iteration=clf.best_iteration)
        imp_df = pd.DataFrame()
        imp_df['feature'] = use_cols
        imp_df['gain'] = clf.feature_importance(importance_type="gain") 
        break
    return imp_df.sort_values("gain", ascending=False)

In [94]:
train_df["type"].unique()

array(['1JHC', '2JHC', '2JHH', '3JHH', '3JHC', '3JHN', '1JHN', '2JHN'],
      dtype=object)

In [97]:
learn(train_df, "1JHC")

Training until validation scores don't improve for 100 rounds.
[100]	training's l1: 0.266387	valid_1's l1: 0.266195
[200]	training's l1: 0.263987	valid_1's l1: 0.265487
[300]	training's l1: 0.262194	valid_1's l1: 0.265109
[400]	training's l1: 0.260875	valid_1's l1: 0.265035
Early stopping, best iteration is:
[321]	training's l1: 0.261806	valid_1's l1: 0.264987


,feature,gain
4,bond_lengths_2_1,7438.598073
6,bond_lengths_3_1,2778.557551
56,bond_count_N,2105.168925
88,mean_dist_in_molecule,2070.333901
29,CN_next_L2dist_max,1963.011333
84,max_dist_in_molecule,1389.205813
46,all_next_L2dist_mean,1355.858896
93,min_dist_in_molecule,1265.858139
70,diff_max_dist_in_molecule,1262.375337
48,all_next_L2dist_std,1259.591222


In [99]:
learn(train_df, "2JHH")

Training until validation scores don't improve for 100 rounds.
[100]	training's l1: 0.0816942	valid_1's l1: 0.0831655
[200]	training's l1: 0.0809033	valid_1's l1: 0.0828081
[300]	training's l1: 0.0803583	valid_1's l1: 0.0826854
[400]	training's l1: 0.0798735	valid_1's l1: 0.0826286
Early stopping, best iteration is:
[389]	training's l1: 0.0798973	valid_1's l1: 0.0825947


,feature,gain
99,rel_min_dist_in_index_atom_0,386.993713
117,H_dist2,259.000407
55,bond_count_H,204.005792
2,bond_lengths_1_1,194.194623
120,N_dist1,193.275278
126,all_dist3,159.243324
60,count_H,144.148968
111,std_dist_in_index_atom_1,129.774813
88,mean_dist_in_molecule,126.228119
83,max_dist_in_index_atom_1,123.624023


In [98]:
learn(train_df, "3JHH")

Training until validation scores don't improve for 100 rounds.
[100]	training's l1: 0.127902	valid_1's l1: 0.130772
[200]	training's l1: 0.125771	valid_1's l1: 0.129834
[300]	training's l1: 0.124628	valid_1's l1: 0.129665
[400]	training's l1: 0.123838	valid_1's l1: 0.12972
Early stopping, best iteration is:
[331]	training's l1: 0.124268	valid_1's l1: 0.129535


,feature,gain
55,bond_count_H,3237.039496
93,min_dist_in_molecule,2772.816539
88,mean_dist_in_molecule,2256.279319
69,diff_max_dist_in_index_atom,2128.690136
84,max_dist_in_molecule,2105.079907
95,rel_max_dist_in_index_atom_1,1860.146054
53,bond_count_C,1611.968806
67,count_in_molecule,1227.527941
114,C_dist3,1215.646159
83,max_dist_in_index_atom_1,1176.435077


In [103]:
train_df[(train_df.target > 1)&(train_df.type=="3JHH")]["bond_count_H"].mean()

16.571844307020736

In [104]:
train_df[(train_df.target < 1)&(train_df.type=="3JHH")]["bond_count_H"].mean()

15.340710228594169